# Расчетное задание 1

## Идентификация сообщений, передаваемых по зашумленному каналу связи

### Часть 1. Последовательная передача одинаковых сообщений

#### Исходный алфавит

In [1]:
alphabet = {}
with open('alphabet.txt', encoding='utf8') as f:
    content = f.readlines()
    content = [x.rstrip().split('\t') for x in content] 
    for pair in content:
        alphabet[pair[0]] = pair[1]
    print(alphabet)

{'6': '0000110', 'З': '0010010', 'л': '0110111', 'Ж': '0010001', 'У': '0011110', 'с': '0111101', 'Ы': '0100110', 'к': '0110110', 'э': '1001001', 'Щ': '0100100', 'ъ': '1001000', 'Э': '0101000', '(': '1010100', '3': '0000011', '9': '0001001', 'Р': '0011011', 'Е': '0001111', 'я': '1001011', 'б': '0101100', 'Ц': '0100001', 'й': '0110101', ' ': '1010110', 'Ь': '0100101', 'г': '0101110', 'С': '0011100', '-': '1010001', '0': '0000000', 'Х': '0100000', 'в': '0101101', 'Н': '0011000', '!': '1001110', 'П': '0011010', 'ю': '1001010', '_': '1010010', 'Ё': '0010000', 'ф': '1000000', ':': '1001111', '4': '0000100', 'ц': '1000010', 'Ч': '0100010', '5': '0000101', 'и': '0110100', '8': '0001000', '1': '0000001', 'ж': '0110010', 'А': '0001010', ',': '1001101', 'е': '0110000', 'о': '0111010', 'щ': '1000101', 'ы': '1000111', 'н': '0111001', 'Ф': '0011111', 'Г': '0001101', 'Л': '0010110', 'ё': '0110001', 'х': '1000001', 'К': '0010101', 'а': '0101011', 'Ш': '0100011', 'Т': '0011101', '2': '0000010', 'Я': '0

In [2]:
def get_char(code: str):
    for key, value in alphabet.items():
        if value == code:
            return key
    #return [k for k, v in dic.items() if v == code][0]

Словарь априорных вероятностей:

In [3]:
apriori = alphabet.copy()
n = len(alphabet)
p = 1 / n
for key, value in apriori.items():
    apriori[key] = p

Функция, находящая максимальное значение в словаре

In [4]:
def find_max(d: dict):
    return max(d, key=d.get)

#### Входные данные

In [5]:
with open('input.txt', encoding='utf8') as f:
    content = f.readlines()

Колличество букв в каждом сообщении равно $s$:

In [6]:
s = int(content[0].split(': ')[1])
print('s = {s}'.format(s=s))

s = 222


Вероятность искажения каждого отдельного символа (бита) равна $q$, вероятность правильной передачи $p = 1 - q$:

In [7]:
q = float(content[2].split(': ')[1])
p = 1 - q
print('p = {p}, q = {q}'.format(p=p, q=q))

p = 0.832, q = 0.168


Колличествно передач равно $m$:

In [8]:
m = int(content[3].split(': ')[1])
print('m = {m}'.format(m=m))

m = 18


Функция вероятности того, что $code1$ и $code2$ - это одно и то же сообщение 

In [9]:
def find_probability(code1, code2):
    assert len(code1) == len(code2)
    probability = 1
    for i in range(len(code1)):
        probability = probability * (p if code1[i] == code2[i] else q)
    return probability

Функция расчета апостариорной вероятности при получении кода $code$:

In [10]:
def posterior_probabilities(dict_char: dict, code: str):
    cond_probs = alphabet.copy()
    for char in cond_probs.keys():
        cond_probs[char] = find_probability(alphabet[char], code)
    
    summ = 0
    for char in cond_probs.keys():
        summ = summ + cond_probs[char] * dict_char[char]
    
    for char in dict_char.keys():
        new_prob = cond_probs[char] * dict_char[char] / summ
        dict_char[char] = new_prob

Функция, возврщающая отсортированный по двоичному коду алфавит.

In [11]:
def get_sorted_keys():
    sorted_values = sorted(alphabet.values())
    sorted_keys = []
    for value in sorted_values:
        sorted_keys.append(get_char(value))
    return sorted_keys

#### Все символы равновероятны

In [12]:
messages = []

for i in range(m):
    codes = content[i + 4].split(': ')[1].rstrip().split(' ')
    messages.append(codes)

In [13]:
import matplotlib.pyplot as plt
from matplotlib import rc
font = {'family': 'Arial', 'weight': 'normal'}
rc('font', **font)

chars = []

for i in range(s):
    char = apriori.copy()
    chars.append(char)

for i in range(m):
    for j in range(s):
        posterior_probabilities(chars[j], messages[i][j])

    curr = chars[7]
    sorted_keys = get_sorted_keys()
    sorted_values = []
    for key in sorted_keys:
        sorted_values.append(curr[key])

    check = 7
    fig = plt.figure(figsize=(11,7))
    plt.bar(range(len(curr)), sorted_values, align='center')
    plt.xticks(range(len(curr)), sorted_keys)
    plt.xlabel('$x$', fontsize=20, labelpad=20)
    plt.ylabel('$P(x)$', fontsize=20, labelpad=20)
    plt.tick_params(labelsize=9, pad=10)
    plt.title('Распределение вероятностей {0}-ой буквы сообщения после {1}-ой посылки'.format(check,i+1), fontsize=15)
    plt.tight_layout()
    plt.xlim([0, len(curr)])
    plt.ylim([0, 1])
    plt.tight_layout()
    plt.savefig('./pics/uniform{i}.png'.format(i=i+1))
    plt.close(fig)
            
    print(i+1, end=': ')
    for char in chars:
        max_char = find_max(char)
        print(max_char, end='')
    print()

1: 8:ЛДьэчимв 4зГЩр,ПизьдпПзж:_235ц1Ё , вжШНН_пк вкБНпыпЯлдчт_зая(т пЯ гХНоиб(вевЯчтЮосйиёцуЗНитЩтисШ (ирелЬаМр3чЦТзШЬо,тча. Ял!Л:тжгЯ(ЯоПсебмеЯТ! про(йЩат6(хУЙкРммвкеифКо ззЬа:и(й?ннелЯр.УА№ц(рюсяЁрн7х кядя-_2!кА(и3елаю эдЗ.
2: б:(гьэ3инв 4пвЩмхюиШ6ЫсУЗжЕ_З3Ьъ9З :ЛвжШмУ_нжывлРСа6пАЪЪ№У жа!(тьпг№тЭооис вкгояУЭоллиЖцуЛЗик4кисЮ (имикйп7вчяеТёавЯ,кцаВ гёя :ЫжДо(ооПребоЁгся прПСиЩат!(фДЗкнННбтеиъ(н(зпваяи(лЗОсжуЧт!УАхХЛбЯсэЁтн3х иадэЭжч.Уэ(с3Хлаю(эгП,
3: г, ДьэчЛмв 4ауЩм, изшгттпз:_23Ьъ5Зь, вЧШмжжно вкорпьпЯлучт жачет по№геориЩьвжрЯятнойтеЖ уЛЗеЫЩтисЮ (имикла рБяЦсзаво,ича. Азя ,тоДо потрертегс: пробе4ать хти птмвтейфй? зРдпчи к(суелЯть 2-ШЛрЯсчетн7х жадэния. ю(сБжлаю ,дП.
4: г:(ДьэчЛов 4адЩм: ил(Ыттпз: Ч35ъ4_6, вжШмжжуо(вкорп пЭлу№т(жачетьпг теориЩьвжрЯятностеЖ уЛЗикЩуинЮ (имикйа СБяХславП,кца, ДляЛэтого потржбтегсю прогешат6 эти простейфй? зБЫачи к сселЯть 2-ШЛрасхетныщ жавЮния. ю(с3жлаю(,гПД
5: г, ДьячЛов 4ауим, из гттпз: 23Ь81_6, вжамжжно вкоро получу зачет по теорииьЮжроятнойтеК у Пик

#### Вероятности задаются ихсодя из частоты встречания букв в русском языке

In [14]:
with open('frequencies.txt', encoding='utf8') as f:
    content = f.readlines()
    frequncies = {}
    content = [x.rstrip().split('\t') for x in content] 
    for pair in content:
        frequncies[pair[0]] = float(pair[1])
    print(frequncies)

{'б': 1.51, 'я': 2.22, 'й': 1.31, 'ц': 0.52, 'ь': 1.9, 'ч': 1.27, 'ю': 1.03, 'м': 3.29, 'л': 4.32, 'ж': 0.78, 'и': 7.45, 'с': 5.45, 'д': 2.56, 'е': 8.1, 'ш': 0.77, 'т': 6.3, 'о': 9.28, 'щ': 0.49, 'р': 5.53, 'н': 6.35, 'к': 3.47, 'п': 3.35, 'ф': 0.4, 'э': 0.17, 'ъ': 0.04, 'ы': 2.11, 'ё': 0.14, 'у': 2.9, 'з': 1.81, 'х': 0.92, 'а': 8.66, 'в': 4.19, 'г': 1.41}


In [15]:
def get_frequency(char: str):
    assert len(char) == 1
    char = char.lower()
    if char in frequncies.keys():
        return 33 * apriori[char] * frequncies[char] / sum(frequncies.values())
    else:
        return apriori[char]

In [16]:
apriori_weight = apriori.copy()
for key in apriori_weight.keys():
    apriori_weight[key] = get_frequency(key)
    
print(apriori_weight)

{'6': 0.011494252873563218, 'З': 0.00686551724137931, 'л': 0.016386206896551726, 'Ж': 0.0029586206896551727, 'У': 0.011, 'с': 0.020672413793103445, 'Ы': 0.008003448275862069, 'к': 0.013162068965517242, 'э': 0.0006448275862068966, 'Щ': 0.0018586206896551724, 'ъ': 0.00015172413793103449, 'Э': 0.0006448275862068966, '-': 0.011494252873563218, '(': 0.011494252873563218, '3': 0.011494252873563218, '9': 0.011494252873563218, 'Р': 0.020975862068965515, 'Е': 0.030724137931034477, 'я': 0.008420689655172415, 'б': 0.005727586206896551, 'й': 0.00496896551724138, ' ': 0.011494252873563218, 'Ь': 0.007206896551724137, 'г': 0.005348275862068965, 'С': 0.020672413793103445, 'Х': 0.0034896551724137932, '0': 0.011494252873563218, 'в': 0.015893103448275863, 'Н': 0.02408620689655172, '!': 0.011494252873563218, 'П': 0.012706896551724139, 'ю': 0.003906896551724138, '_': 0.011494252873563218, 'Ё': 0.0005310344827586207, 'ф': 0.0015172413793103448, ':': 0.011494252873563218, '4': 0.011494252873563218, 'ц': 0.00

In [17]:
chars = []

for i in range(s):
    char = apriori_weight.copy()
    chars.append(char)
    
for i in range(m):
    for j in range(s):
        posterior_probabilities(chars[j], messages[i][j])
          
            
    curr = chars[7]
    sorted_keys = get_sorted_keys()
    sorted_values = []
    for key in sorted_keys:
        sorted_values.append(curr[key])
        
    check = 7
    fig = plt.figure(figsize=(11,7))
    plt.bar(range(len(curr)), sorted_values, align='center')
    plt.xticks(range(len(curr)), sorted_keys)
    plt.xlabel('$x$', fontsize=20, labelpad=20)
    plt.ylabel('$P(x)$', fontsize=20, labelpad=20)
    plt.tick_params(labelsize=9, pad=10)
    plt.title('Распределение вероятностей {0}-ой буквы сообщения после {1}-ой посылки'.format(check,i+1), fontsize=15)
    plt.tight_layout()
    plt.xlim([0, len(curr)])
    plt.ylim([0, 1])
    plt.tight_layout()
    plt.savefig('./pics/weight{i}.png'.format(i=i+1))
    plt.close(fig)
    
    print(i+1, end=': ')
    for char in chars:
        max_char = find_max(char)
        print(max_char, end='')
    print()

1: 8:ЛДь9чимв 4зЕирТПизьдпПзо:_235_1е Т воаНН_пк вкАНпыпЯлдчт_заяит пЯ геНоиб(вевЯчтаосиие_уЗНититиса СирелЬаМр3ч1ТзаЬо,тча. Ял!Л:тогЯ(ЯоПсебмеЯТ! проСииат6(хУикРммвкеи?Ко ззЬа:и(и?ннелЯр. А№Ч(рЯсяерн7х када-_2!кА(и3елаА 9дЗ.
2: А:ЛА6э3инв ВаЕХр:_изЛдооозЕ_235А9_ :ЛвоШНо_но виоНаАпАлл№т за!ет по_теооии(вето№тноТЕеЖИуЛНетиоиса )иреллаЛТ3:еТИаво,очаВ Ал:Л:тоДо(ооореиоеоТ: проре4ат!(-текОоовтеи.ио(зова:и(иеОоелЯт! А-еЛрЯТяетнИх еадаееА.тА(и3елаю(,тП,
3: !, Дь,чЛмв 4ауим, изшгттпз:_23Ь85Зь, вЧаможно вкорпьпЯлучт еачет по№георииьверЯятнойтеЖМуЛЗеЫитиса (имикла рЕяЦсзавовича. Азя ,тоДо потрертегс: пробе4ать хти птмвтеифие зРдпчи к(суелЯть 2-ШЛрЯсчетн7х оадания. Я(сАелаю ,дП.
4: гЕ(Д!ячЛов Вауим: ил(Ыттпз:_23584_6, вЧамжжно влоро полу№т(зачет по теории(вероятностеК уЛПекЩтина Кирилла ряяХсИавовича, АляЛ,тоДо потреитетс: проте4ат6 хти птостейшие задачи и сселАть 2-3ЛраТчетн7х завания. а(саелаю(.тПВ
5: г, ДьячЛов 4ауим, из гттпз: 23Ь81_6, воаможно вкоро получу зачет по теорииьЮероятнойтеК у Пик